In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.model_selection import cross_val_score,StratifiedKFold,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix,precision_score,f1_score,recall_score

In [3]:
results = pd.read_csv(r'data\results.csv')
races = pd.read_csv(r'data\races.csv')
quali = pd.read_csv(r'data\qualifying.csv')
drivers = pd.read_csv(r'data\drivers.csv')
constructors = pd.read_csv(r'data\constructors.csv')
circuit = pd.read_csv(r'data\circuits.csv')

In [5]:
df1 = pd.merge(races,results,how='inner',on=['raceId'])
df2 = pd.merge(df1,quali,how='inner',on=['raceId','driverId','constructorId'])
df3 = pd.merge(df2,drivers,how='inner',on=['driverId'])
df4 = pd.merge(df3,constructors,how='inner',on=['constructorId'])
df5 = pd.merge(df4, circuit, how='inner', on=['circuitId'], suffixes=('_df4', '_circuit'))


In [9]:
print(df5.shape)
print(df5.columns)

(8403, 51)
Index(['raceId', 'year', 'round', 'circuitId', 'name_x', 'date', 'time_x',
       'url_x', 'resultId', 'driverId', 'constructorId', 'number_x', 'grid',
       'position_x', 'positionText', 'positionOrder', 'points', 'laps',
       'time_y', 'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId', 'qualifyId', 'number_y', 'position_y',
       'q1', 'q2', 'q3', 'driverRef', 'number', 'code', 'forename', 'surname',
       'dob', 'nationality_x', 'url_y', 'constructorRef', 'name_y',
       'nationality_y', 'url_df4', 'circuitRef', 'name', 'location', 'country',
       'lat', 'lng', 'alt', 'url_circuit'],
      dtype='object')


In [16]:
# List of columns to drop
columns_to_drop = ['round','circuitId','time_x','url_x','resultId','driverId','constructorId','number_x','positionText','position_x','positionOrder','laps','time_y','rank','fastestLapTime','fastestLapSpeed','qualifyId','driverRef','number','code','url_y','circuitRef','location','lat','lng','alt','number_y','points','constructorRef','name_x','raceId','fastestLap','q2','q3','milliseconds','q1']
columns_to_drop = [col for col in columns_to_drop if col in df5.columns]

# Drop the columns
data = df5.drop(columns=columns_to_drop)


In [17]:
data.columns

Index(['year', 'date', 'grid', 'statusId', 'position_y', 'forename', 'surname',
       'dob', 'nationality_x', 'name_y', 'nationality_y', 'url_df4', 'name',
       'country', 'url_circuit'],
      dtype='object')

In [18]:
import F1PyStats as fp

ModuleNotFoundError: No module named 'F1PyStats'

In [15]:
data.columns

Index(['raceId', 'year', 'round', 'circuitId', 'name_x', 'date', 'time_x',
       'url_x', 'resultId', 'driverId', 'constructorId', 'number_x', 'grid',
       'position_x', 'positionText', 'positionOrder', 'points', 'laps',
       'time_y', 'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId', 'qualifyId', 'number_y', 'position_y',
       'q1', 'q2', 'q3', 'driverRef', 'number', 'code', 'forename', 'surname',
       'dob', 'nationality_x', 'url_y', 'constructorRef', 'name_y',
       'nationality_y', 'url_df4', 'circuitRef', 'name', 'location', 'country',
       'lat', 'lng', 'alt', 'url_circuit'],
      dtype='object')